In [1]:
# Install the latest version of the OpenAI package
!pip install --upgrade openai

In [2]:
# Import necessary libraries
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

In [3]:
# Load environment variables from a .env file
load_dotenv(find_dotenv())

True

In [73]:
# Initialize the OpenAI client with the API keysk-proj-9Gw8RhdBekIhLh05pGrKT3BlbkFJRhweDVVTVC9C85RryRBJ
api_key = 'sk-proj-9Gw8RhdBekIhLh05pGrKT3BlbkFJRhweDVVTVC9C85RryRBJ'
client = OpenAI(api_key=api_key)


In [74]:
# Define tools for the assistant
tools = [
    {
        "function": {
            "name": "get_weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": [
                            "c",
                            "f"
                             ],
                        "description": "The temperature unit to use. Infer this from the user's location."
                    }
                },
                "required": ["location"]
            },
            "description": "Get the current weather"
        },
        "type": "function"
    }
    
]

In [75]:
# Create a new assistant
weather_helper = client.beta.assistants.create(
    model='gpt-4',
    name='AK-Functions-Notebook',
    instructions="You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    tools=tools,
    description='This is an assistant created from a notebook.'
)
print(weather_helper.model_dump_json(indent=2))
# "id": "asst_RYwnAtfa5yjRGzWCHhp3boQv"

{
  "id": "asst_RYwnAtfa5yjRGzWCHhp3boQv",
  "created_at": 1723586565,
  "description": "This is an assistant created from a notebook.",
  "instructions": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
  "metadata": {},
  "model": "gpt-4",
  "name": "AK-Functions-Notebook",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "description": "Get the current weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
     

In [76]:
# Create a new thread
thread = client.beta.threads.create()

In [77]:
thread

Thread(id='thread_cFkhQHu6JRClICqgfJR15Y3s', created_at=1723586569, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [78]:
# Predefined weather data
weather_data = {
    'New York': 25,
    'Mumbai': 34,
    'Jaipur': 21,
    'Manali': 5
}

In [79]:
user_input="What is the capital of india ?"

In [80]:
# # Start the interactive session
# while True:
#     # Get user input
#     user_input = input("Enter your query (e.g., 'What is the weather today in Ahmedabad, Gujarat?') or 'exit' to end: ")

#     if user_input.lower() == 'exit':
#         print("Ending the session.")
#         break

In [81]:
thread.id, user_input

('thread_cFkhQHu6JRClICqgfJR15Y3s', 'What is the capital of india ?')

In [82]:
# Create a user message
message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

In [83]:
message

Message(id='msg_ykkluVCibAGelDo9AOwOMGAr', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the capital of india ?'), type='text')], created_at=1723586585, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_cFkhQHu6JRClICqgfJR15Y3s')

In [84]:
# Create a run for the assistant
run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id='asst_RYwnAtfa5yjRGzWCHhp3boQv',
        instructions="Please address the user as Akshat."
    )

In [85]:
run.status

'queued'

In [86]:
 # Retrieve the run details to get the tool_call_id
retrieve_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [87]:
retrieve_run.status

'completed'

In [88]:
 # List the thread messages
thread_messages = client.beta.threads.messages.list(thread.id)
thread_messages.data

[Message(id='msg_SK0KGh9gqBzREm0AzOvfqku7', assistant_id='asst_qpKLh5PBrRBMbz04Wv8M2WdN', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The capital of India is New Delhi, Akshat.'), type='text')], created_at=1723586591, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_peoIGD1ETGUZp6Kl4ohBy1T4', status=None, thread_id='thread_cFkhQHu6JRClICqgfJR15Y3s'),
 Message(id='msg_ykkluVCibAGelDo9AOwOMGAr', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the capital of india ?'), type='text')], created_at=1723586585, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_cFkhQHu6JRClICqgfJR15Y3s')]

In [89]:
user_messages = ''
assistant_messages = ''

for tm in reversed(thread_messages.data):
        content = tm.content[0].text.value
        if tm.role == 'user':
            user_messages += content + '\n'
        if tm.role == 'assistant':
            assistant_messages += content + '\n'
            
print("User's query:")
print(user_messages.strip())

print("Assistant's response:")
print(assistant_messages.strip())

User's query:
What is the capital of india ?
Assistant's response:
The capital of India is New Delhi, Akshat.


In [90]:
retrieve_run

Run(id='run_peoIGD1ETGUZp6Kl4ohBy1T4', assistant_id='asst_qpKLh5PBrRBMbz04Wv8M2WdN', cancelled_at=None, completed_at=1723586591, created_at=1723586589, expires_at=None, failed_at=None, incomplete_details=None, instructions='Please address the user as Akshat.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1723586589, status='completed', thread_id='thread_cFkhQHu6JRClICqgfJR15Y3s', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_weather', description='Get the current weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f'], 'description': "The temperature unit to use. Infer this from the user's location."}}, 'required': ['location']}, strict=False), type='function')], tru

In [91]:
retrieve_run.required_action

In [92]:
submit_tool_outputs = retrieve_run.required_action.submit_tool_outputs
print(submit_tool_outputs)


AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [55]:
submit_tool_outputs.tool_calls[0].function.arguments
import json
temp=json.loads(submit_tool_outputs.tool_calls[0].function.arguments)

NameError: name 'submit_tool_outputs' is not defined

In [69]:
weather_output=f"The weather for {temp.get('location')} is {weather_data.get(temp.get('location'))}"
weather_output

NameError: name 'temp' is not defined

In [70]:
weather_output

NameError: name 'weather_output' is not defined

In [71]:
submit_tool_outputs.tool_calls[0].function.name

NameError: name 'submit_tool_outputs' is not defined

In [72]:
tool_call_id=submit_tool_outputs.tool_calls[0].id
tool_call_id

NameError: name 'submit_tool_outputs' is not defined

In [ ]:
# Submit the tool output
client.beta.threads.runs.submit_tool_outputs(
                run_id=run.id,
                thread_id=thread.id,
                tool_outputs=[
                    {
                        "tool_call_id": tool_call_id,
                        "output": weather_output,
                    }
                ]
            )